# Fonctions SparkSQL

SparkSQL fournit un ensemble de fonctions intégrées utilisables avec les dataframes. Ces fonctions ont deux avantages :
 * Les fonctions intégrées sont généralement optimisées par le moteur Catalyst pour une meilleure performance.
 * Les fonctions intégrées sont disponibles dans toutes les installations Spark, ce qui facilite le partage et la portabilité du code.

Références :
 * Scala API : https://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/functions$.html
 * SQL : https://spark.apache.org/docs/latest/api/sql/index.html

## Préambule

In [ ]:
import $ivy.`org.apache.spark::spark-core:3.4.1`
import $ivy.`org.apache.spark::spark-sql:3.4.1`
import $ivy.`org.slf4j:slf4j-reload4j:2.0.6`

import org.apache.logging.log4j.Level
import org.apache.logging.log4j.core.config.Configurator

// Avoid disturbing logs
Configurator.setRootLevel(Level.OFF)

In [ ]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd._

val spark = {
  NotebookSparkSession.builder()
    .progress(enable = true, keep = true, useBars = true)
    .master("local[*]")
    // L'appel ci-dessous sert à donner un nom à votre application
    // Ce apparaîtra notamment dans la Spark UI
    .appName("SparkSQL - Fonctions")
    .getOrCreate()
}

import spark.implicits._

// Ce script fournit que élément supplémentaires pour rendre l'affichage plus confortable
import $file.^.internal.spark_helper
import spark_helper.implicits._

## Tweets

Pour cet atelier, nous allons utiliser un dataser contenant des tweets au format JSON, suite à une recherche sur l'expression "big data" et datant de 2021.

Note : une partie de la requête de chargement consiste à ne conserver que les lignes valides (`.where($"_corrupt_record".isNull)`).

In [ ]:
%%data

val tweets =
  spark.read
    .json("data/tweets.json.gz")
    .cache()
    .where($"_corrupt_record".isNull)
    .drop("_corrupt_record")

tweets.createOrReplaceTempView("tweets")

tweets

Ci-dessous, nous pouvons constater que le [schéma des tweets](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet) est imposant et contient diverses sous-structures.

Note : sur l'affichage ci-dessous, n'hésitez pas à cliquer-droit sur l'affichage du schéma et à sélectionner "Enable Scrolling for Outputs".

In [ ]:
tweets.printSchema()

## Les langues représentées

Dans chaque tweet, il y a un champ `lang`, qui indique la langue dans laquelle a été écrit le tweet. La valeur de ce champ est décrite par la norme BCP47 ([liste des langues](http://www.iana.org/assignments/language-subtag-registry/language-subtag-registry)).

Donner la liste des langues utilisées dans les tweets, en les classant de la plus utilisée à la moins utilisée. Faites intervenir pour cela les méthodes `.groupBy()`, `.count()` et `.orderBy()`.

Pour rappel, l'accès à un champ se fait avec la notation `$"field_name"`. Pour classer des lignes de manière décroissante par rapport à une colonne, utilisez la méthode `.desc` sur cette colonne.

In [ ]:
%%data

val result = ???

result.explain()
result

**Ce qu'il faut voir**

La fonction `count` apparaît sous deux formes dans le plan d'exécution, dans l'ordre :
 * `HashAggregate(keys=[lang#29], functions=[partial_count(1)])`
 * `HashAggregate(keys=[lang#29], functions=[count(1)])`

Ce plan montre que Spark va organiser le décompte des lignes en deux phases. Dans une première phase, chaque exécuteur va compter de son côté le nombre de lignes par langue (`partial_count`). Puis, les décomptes partiels sont redistribués selon un _hash_ calculé sur la langue (`Exchange hashpartitioning(lang#3676, 200), ENSURE_REQUIREMENTS, [plan_id=2830]`), afin d'avoir les décomptes partiels d'une même langue au sein du même exécuteur. Le traitement se termine alors par la somme des décomptes partiels par langues (`count`).

## Utilisateurs

Dans un tweet, un utilisateur est représenté par une sous-structure contenant plusieurs informations, dont l'ID, le nom, le site Web...

Donner la liste du _screen_name_ des utilisateurs apparaissant dans les tweets, en les classant de l'utilisateur ayant le plus tweeté à l'utilisateur ayant le moins tweeté.

Note : Pour accéder à un champ `b` d'une sous-structure `a`, vous devez utiliser la syntaxe `$"a.b"`.

In [ ]:
%%data

val result = ???

result

## Hashtags

Les tweets peuvent contenir des hashtags, qui sont des mots apparaissant dans le texte du tweet précédé d'un _hash_ (`#`).

Dans la structure du tweet, les hashtags apparaissent dans le champ `entities`, qui est une structure. Cette structure contient le champ `hashtags`. Ce champ `hashtags` est un _array_ (ou liste). Celui-ci un champ `text`.

Voici un extrait du schéma qui nous intéresse :

```
 |-- entities: struct (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
```

Nous voulons la liste des hashtags les plus utilisés.

Il est possible de vérifier si un tweet contient des hashtags. Pour cela, vous pouvez utiliser l'opération `.where()` et utiliser la fonction `size()` sur le champ `entities.hashtags`.

Pour décomposer une liste, vous pouvez utiliser la fonction `explode()` sur le champ `$"entities.hashtags"` dans un `.select()`. Cette fonction crée une nouvelle pour chaque élément contenu dans la liste.

In [ ]:
%%data

val result = ???

result.explain()
result

## Hashtag le plus utilisé

Vous allez maintenant fournir le hashtag le plus utilisé dans le dataset `tweets`.

Pour récupérer la première ligne d'un dataframe, vous pouvez utiliser la méthode `.first()`. Celle-ci retourne une valeur de type `Row`. Rechercher dans la documentation SparkSQL pour comprendre comment extraire un champ de cet objet.

In [ ]:
val hashtag: String = ???

println(hashtag)

In [ ]:
%%data

tweets.rollup("lang", "user.screen_name").count().orderBy($"count".desc)

## Ensemble des interactions entre utilisateurs

Nous allons nous intéresser aux interactions visibles entre utilisateurs sur Twitter.

Un tweet contient un `quoted_status` et un champ `retweeted_status`. Ces deux champs indiquent respectivement si un tweet cite un autre tweet ou si un tweet est un retweet d'un autre. Ces champs sont des sous structures avec un schéma relativement identiques. Nous trouvons notamment le champ `user`, contenant le champ `screen_name`, indiquant l'utilisateur du tweet d'origine.

Il y a aussi un champ `in_reply_to_screen_name` à la racine du tweet, qui, dans le cas d'un tweet de réponse à un autre, indique l'utilisateur qui a émis le tweet initial.

 1. Dans l'exercice ci-dessous, vous allez caractériser le _screen name_ de l'utilisateur à l'origine de chaque type d'interaction (citation, retweet, réponse) (`from_user`) et le _screen name_ de l'utilisateur destinataire de l'interaction (`to_user`). Par exemple, si un utilisateur A cite le tweet de l'utilisateur B, `from_user` correspond à A et `to_user` correspond à B.
 2. Rassemblez toutes les interactions dans un seul dataframe, en utilisant l'opération `.union()`
 3. Compter à la fois les interactions individuelles, le total des interactions émises par utilisateur, le total des interactions reçues par utilisateur et le total des interactions.

Pour ce dernier point, utilisez la méthode `.cube(col_1, col_2, ...)`. Cette méthode est une généralisation de `.groupBy()`, sachant qu'elle effectue un regroupement selon toutes les combinaisons possibles entre les colonnes passées en paramètre (eg. col_1 et col_2, col_1 seule, col_2 seule, et sur l'ensemble du dataframe).

In [ ]:
val replyInteractions = ???

val quotedInteractions = ???

val retweetInteractions = ???

val allInteractions = ???

val interactionCount = ???

interactionCount.explain()
interactionCount.orderBy($"count".desc).showHTML(truncate=480)

## Tranches horaires les plus actives

Donnez la moyenne sur l'ensemble du dataset du nombre de tweets envoyés pour chaque tranche horaire. Le champ à utiliser est le champ `created_at` exprimé en millisecondes depuis EPOCH.

Dans cet exercice, vous allez avoir besoin des fonctions suivantes :
 * `to_timestamp(seconds)` : convertis un nombre (de type Long) de secondes depuis EPOCH en valeur de type Timestamp.
 * `date_trunc(section, timestamp)` : tronque un timestamp au niveau d'une section donné (`"year"`, `"day"`, `"hour"`...).
 * `hour(timestamp)` : retourne uniquement la section "heure" d'un timestamp.
 * `avg(colonne)` : retourne la moyenne sur une colonne, suite à un regroupe (eg. `.groupBy()`). Il s'agit d'une fonction d'agrégation. Elle ne peut s'utiliser que dans la méthode `.agg()`.

In [ ]:
val result = ???

result.showHTML(limit=24,truncate=40)
result.explain()

## User-Defined Function (UDF)

Vous avez la possibilité de définir vos propres fonctions SparkSQL. Ces fonctions sont alors nommées des _User-Defined Function_ ou UDF.

Cet exercice est divisé en deux parties. La première partie est juste dessous. Elle consiste à utiliser les fonctions SparkSQL intégrées pour calculer un timestamp et en extraire l'heure.

Regarder comment sont représenter les fonction dans le plan d'exécution.

In [ ]:
%%data

def timestamp_hour(c: Column): Column =
  hour(to_timestamp(c))

val result =
  tweets
    .where($"created_at".isNotNull)
    .select(
      $"created_at",
      to_timestamp($"created_at" / 1000).as("timestamp"),
      timestamp_hour($"created_at" / 1000).as("hour")
    )

result.explain()
result

Nous allons faire la même chose ci-dessous, mais cette fois en utilisant des UDF.

Pour créer une UDF, vous devez utiliser la fonction `udf[Out, In](f).withName("<udf_name>")`, où `Out` est le type de sortie, `In` est le type d'entrée de la fonction, `f` est la fonction qui sera utilisée par l'UDF et `<udf_name>` est le nom de la fonction qui apparaîtra dans le plan d'exécution.

In [ ]:
%%data

import java.time._
import java.sql.Timestamp

def toTimestamp(epochMilli: Long): Timestamp = {
  new Timestamp(epochMilli)
}

def hourOfEpochMilli(epochMilli: Long): Int = {
  val dateTime = toTimestamp(epochMilli).toLocalDateTime()
  dateTime.getHour()
}

val toTimestamp_udf = ???
val hourOfEpochMilli_udf = ???

val result =
  tweets
    .where($"created_at".isNotNull)
    .select(
      $"created_at",
      toTimestamp_udf($"created_at").as("timestamp"),
      hourOfEpochMilli_udf($"created_at").as("hour")
    )

result.explain()
result

Vous allez maintenant faire la même chose, mais cette fois en utilisant le langage SQL.

Pour pouvoir utiliser une UDF dans une requête SQL, il d'abord enregistrer l'UDF. Pour cela, il faut utiliser la méthode `spark.udf.register("<nom_SQL>", udf)`.

In [ ]:
???

val result =
  spark.sql("""
SELECT created_at, toTimestamp(created_at) AS timestamp, hourOfEpochMilli(created_at) AS hour
FROM tweets
""")

result.showHTML(limit=10, truncate=40)
result.explain()